# Inverse Kinematics for 2R robot with Jacobian

Reference trajectory for end-effector is a circle in plane. We have to make an inverse kinematics and find Theta_1 and Theta_2. Also make a PD controller

Import necessery libraries

In [1]:
import mujoco
import mujoco_viewer
import matplotlib.pyplot as plt
import numpy as np

Create model and data

In [2]:
model = mujoco.MjModel.from_xml_path('2r_robot_PID.xml')
data = mujoco.MjData(model)

Create a FUNCTION 'set_torque' where we use PD controller for our joints actuators.

In [3]:
def set_torque(mj_data, KP, KV, theta_1, theta_2): 
    data.ctrl[0] = KP * (-mj_data.qpos[0] + theta_1) + KV * (0 - mj_data.qvel[0])
    data.ctrl[1] = KP * (-mj_data.qpos[1] + theta_2) + KV * (0 - mj_data.qvel[1])

Set initial angles and links' length. Values in radians (rad). Note that 1rad=pi=180 deg

In [4]:
theta_1 = 1.9 # initial angle of first joint
theta_2 = -2.42 # initial angle of second joint
data.joint('joint_1').qpos[0] = theta_1 
data.joint('joint_2').qpos[0] = theta_2 
l_1 = 1 # first link lenght
l_2 = 1 # second link lenght

Reference circle movement

In [5]:
SIMEND = 10
TIMESTEP = 0.002
STEP_NUM = int(SIMEND/TIMESTEP)

center_circle = [0, 0]
circle_radius = 0.5
phi = np.linspace(0, 2*np.pi, STEP_NUM)

x_ref = center_circle[0] + circle_radius * np.cos(phi)
z_ref = center_circle[1] + circle_radius * np.sin(phi)

Empty lists for data

In [6]:
EE_position_x = [] # empty lists for actual position of end-effector
EE_position_z = []

Make a viewer. Into FOR loop find Theta_1 and Theta_2 using Jacobian

In [7]:
viewer = mujoco_viewer.MujocoViewer(model, data)

for i in range(STEP_NUM):
    if viewer.is_alive:
        
        # Compute Jacobian (J)
        position_EE = data.site_xpos[0]
        jacp = np.zeros((3, 2)) # 3 is for XYZ, 2 is for theta_1 and theta_2
        mujoco.mj_jac(model, data, jacp, None, position_EE, 2)
        J = jacp[[0,2],:] # We leave only the first and third lines associated with X and Z
        print(J)

        # Compute inverse Jacobian (J_inv)
        J_inv = np.linalg.inv(J)

        # Compute dX
        dX = np.array([x_ref[i] - position_EE[0], z_ref[i] - position_EE[2]])

        # Compute dq = J_inv*dX
        dq = J_inv.dot(dX)

        # Update theta_1 and theta_2
        theta_1 += dq[0]
        theta_2 += dq[1]

        data.joint('joint_1').qpos = theta_1
        data.joint('joint_2').qpos = theta_2

        i += 1

        EE_position_x.append(position_EE[0]) # ACTUAL end-effector X position in xpos (it is a zero element in a nested list / list into list)
        EE_position_z.append(position_EE[2]) # ACTUAL z position

        mujoco.mj_step(model, data)
        viewer.render()
    else:
        break

viewer.close()

[[0. 0.]
 [0. 0.]]


LinAlgError: Singular matrix

: 